In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

#We may want these at some point for transforming our output:
#from scipy.special import logit, expit

pd.set_option('display.max_columns', None) #forces all columns to be displayed
pd.set_option('display.max_rows', None) #forces all rows to be displayed

In [3]:
filepath = r'../data/data-reduced-train.csv'
data = pd.read_csv(filepath)

In [37]:
features = data.columns[4:].tolist()
target = data.columns[3]

target_and_feature_list = [target] + features

In [39]:
#extract correlations with the target variable
correlations_with_target = data[target_and_feature_list].corr().iloc[:, 0]
correlations_with_target

% Adults with Diabetes                                 1.000000
% Adults Reporting Currently Smoking                   0.673060
% Adults with Obesity                                  0.683599
Food Environment Index                                -0.634980
% Physically Inactive                                  0.849223
% With Access to Exercise Opportunities               -0.308404
% Excessive Drinking                                  -0.601580
% Driving Deaths with Alcohol Involvement             -0.082421
% Uninsured                                            0.490803
Dentist Rate                                          -0.229810
% with Annual Mammogram                               -0.471106
% Vaccinated                                          -0.376029
% Completed High School                               -0.751547
% Unemployed                                           0.402164
% Children in Poverty                                  0.825812
Income Ratio                            

In [47]:
features_low_corr = [key for key in correlations_with_target.index if np.abs(correlations_with_target[key]) <= 0.3]

In [48]:
data_high_corr = data.drop(columns=features_low_corr)

In [51]:
data_high_corr.head()

,FIPS,State,County,% Adults with Diabetes,% Adults Reporting Currently Smoking,% Adults with Obesity,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Uninsured,% with Annual Mammogram,% Vaccinated,% Completed High School,% Unemployed,% Children in Poverty,Income Ratio,% Children in Single-Parent Households,Average Daily PM2.5,% Severe Housing Problems,% Food Insecure,% Insufficient Sleep,School Funding Adequacy,Median Household Income,% Voter Turnout,% Census Participation,% Homeowners,% Households with Broadband Access,% Black,% Non-Hispanic White
0,17027,Illinois,Clinton,8.7,16.7,34.8,9.0,25.8,63.809317,18.658612,6.070879,48.0,43.0,92.001064,2.951476,9.2,3.919867,16.250475,8.8,9.181226,7.0,32.7,-3699.384446,79612.0,65.471311,73.9,79.555648,87.890706,3.243111,91.007613
1,42071,Pennsylvania,Lancaster,8.2,16.7,35.2,8.8,23.4,80.948635,17.168046,11.306447,53.0,60.0,86.957844,3.524487,11.9,3.779021,15.501977,11.1,14.051126,7.4,36.8,5941.193063,82434.0,69.184565,75.9,69.948992,85.387678,3.722587,80.193630
2,46003,South Dakota,Aurora,8.2,17.0,38.8,7.8,23.1,3.349108,21.089477,13.614801,52.0,31.0,91.299304,1.877934,14.7,3.314294,15.384615,4.7,5.333333,6.0,29.6,-1155.780000,61013.0,73.560209,55.1,76.638177,78.252612,0.907441,85.880218
3,46027,South Dakota,Clay,9.2,16.3,35.6,7.6,22.2,85.568250,22.062377,13.246101,50.0,60.0,93.400600,2.141792,17.1,5.000000,24.854142,5.8,19.581749,9.8,28.4,-629.344700,55263.0,50.536585,65.7,51.089365,89.844904,1.760471,86.302356
4,13205,Georgia,Mitchell,15.9,22.8,42.2,6.8,34.5,59.917260,13.354151,17.431074,50.0,36.0,83.451327,3.968447,32.8,6.162685,53.611394,9.7,17.456359,15.0,41.0,-17591.665000,45296.0,53.574417,51.7,63.604240,77.801615,45.870430,46.126160


In [54]:
len(data_high_corr.columns)

30